In [1]:
import numpy as np
import pandas as pd
import os
import statsmodels.api as sm

In [2]:
def Data_Analysis():
    # input data file name
    train_data=input("Enter the name of training data file [(ex) veh.dat] : ") # data name
    test_data=input("Enter the name of test data file [(ex) vehtest.dat] : ") # data name

    coding_fm=int(input("Select the data Seperator(1 = ' ' or 2 = ','): ")) # data separator of training data
    separator_fm={coding_fm ==1 : " "}.get(True, ",")

    # input header & assign response variables
    header=input("Does the data have column header? (y/n) : ")
    if(header=="y") :
        trdata=pd.read_csv(train_data, sep=separator_fm) # loading data with header
        tstdata = pd.read_csv(test_data,sep = separator_fm)
        res_pos = input(f'Enter the column name of response variable among {list(trdata.columns)} : ')

        y_tr = trdata[res_pos]
        X_tr = trdata.drop(res_pos, axis = 1)

        y_tst = tstdata[res_pos]
        X_tst = tstdata.drop(res_pos, axis = 1)

    else : 
        trdata=pd.read_csv(train_data, sep=separator_fm, header=None) # loading data without header
        tstdata=pd.read_csv(test_data, sep=separator_fm, header=None)
        res_pos = int(input(f'Enter the column position of the response variable : \n [from 1 to {trdata.shape[1]}] : '))
        X_index = []
        for i in range(len(trdata.columns)): 
            if i == res_pos-1 : continue
            X_index.append(i)

        # define response variable and predictor
        y_tr = trdata.iloc[:,res_pos-1]
        X_tr = trdata.iloc[:,X_index]
        y_tst = tstdata.iloc[:,res_pos-1]
        X_tst = tstdata.iloc[:,X_index]


    # input method
    method = input("what kind of method do you want to run? (1 = LDA Classification, 2 = Linear Regression) : ")

    # input name of output file
    out_name=input("Enter the output file name to export [(ex) result.txt] : ")
    
    if method == '1':
        print ('\n\nLDA Classification Method is selected')
        type_y = np.sort(y_tr.unique())
        nx, n, S = {}, [], []
        for i in range(len(type_y)):
            nx[i] = X_tr[y_tr == type_y[i]]
            n.append(len(nx[i]))
            S.append(nx[i].cov())

        # set prior probability 
        prior = n / np.array(n).sum()
        # calculate pooled sample covariance
        S_p = []
        for i in range(len(n)):
            S_p.append(S[i] * (n[i] -1))
        S_p = np.array(S_p).sum(axis=0) / (np.array(n).sum() - len(type_y))
        
        # calculate  d_tst
        d = pd.DataFrame()
        coef = pd.DataFrame()
        constant = []
        for i in range(len(n)):
            coef[i] = nx[i].mean(axis = 0).dot(np.linalg.inv(S_p))
            constant.append((nx[i].mean().dot(np.linalg.inv(S_p)).dot(nx[i].mean()))/ (-2) + np.log(prior[i]))
            d[i] = coef[i].dot(np.array(X_tr.transpose())) + constant[i]
            
        # predict Y
        pred_y = []
        for i in range(len(X_tr)):
            pred_y.append(type_y[d.iloc[i] == d.iloc[i].max()][0])
            
         # result     
        result = pd.DataFrame()
        result['Actual Class'] = y_tr
        result['Predicted Class'] = pred_y



        # confusion matrix
        C = []
        for i in range(len(type_y)):
            for j in range(len(type_y)):
                C.append(((result['Actual Class'] == type_y[i]) & (result['Predicted Class'] == type_y[j])).sum())

        C1 = pd.DataFrame(np.array(C).reshape(len(type_y),len(type_y)),
                          index = type_y, columns = type_y)

        # accuracy for traing data
        acc_tr = round(np.diag(C1).sum()/C1.sum().sum(),3)

        # calculate  d_tst
        d = pd.DataFrame()
        coef = pd.DataFrame()
        constant = []

        for i in range(len(n)):
            coef[i] = nx[i].mean(axis = 0).dot(np.linalg.inv(S_p))
            constant.append((nx[i].mean().dot(np.linalg.inv(S_p)).dot(nx[i].mean()))/ (-2) + np.log(prior[i]))
            d[i] = coef[i].dot(np.array(X_tst.transpose())) + constant[i]

        # predict Y & make result matrix

        pred_y = []
        for i in range(len(X_tst)):
            pred_y.append(type_y[d.iloc[i] == d.iloc[i].max()][0])


        # result     
        result = pd.DataFrame()
        result['Actual Class'] = y_tst
        result['Predicted Class'] = pred_y
        result



        # confusion matrix
        C = []
        for i in range(len(type_y)):
            for j in range(len(type_y)):
                C.append(((result['Actual Class'] == type_y[i]) & (result['Predicted Class'] == type_y[j])).sum())

        C2 = pd.DataFrame(np.array(C).reshape(len(type_y),len(type_y)),
                          index = type_y, columns = type_y)

        # accuracy for Test data
        acc_tst = round(np.diag(C2).sum()/C2.sum().sum(),3)
        acc_tst
        
        # make dummy file
        C1.to_csv('temp1.txt', sep = '\t')

        temp1 = []
        temp1.append('\tPredicted Class \n')
        r = open('temp1.txt','r')
        for line in r.readlines():
            temp1.append(line)
        r.close()
        for i in range(len(temp1)):
            if i == 2:
                temp1[i] = 'Actual \t' + temp1[i] + '\n'
            elif i == 3:
                temp1[i] = 'Class \t' + temp1[i] + '\n'
            else:
                temp1[i] = '\t' + temp1[i] + '\n'

        C2.to_csv('temp2.txt', sep = '\t')

        temp2 = []
        temp2.append('\tPredicted Class \n')
        r = open('temp2.txt','r')
        for line in r.readlines():
            temp2.append(line)
        r.close()    
        for i in range(len(temp2)):
            if i == 2:
                temp2[i] = 'Actual \t' + temp2[i]+ '\n'
            elif i == 3:
                temp2[i] = 'Class \t' + temp2[i]+ '\n'
            else:
                temp2[i] = '\t' + temp2[i]+ '\n'



        # write file
        f = open(out_name,'w+')
        f.write('Confusion Matrix (Training)\n')
        f.write('----------------------------------\n')
        for line in temp1:
            f.write(line)
        f.write('\n')
        f.write('Model Summary (Training)\n')
        f.write('------------------------------\n')
        f.write(f'Overall accuracy = {acc_tr}\n')
        f.write('\n')
        f.write('Confusion Matrix (Test)\n')
        f.write('---------------------------------\n')
        for line in temp2:
            f.write(line)
        f.write('\n')
        f.write('Model Summary (Test) \n')
        f.write('---------------------------------\n')
        f.write(f'Overall accuracy = {acc_tst}')

        f.close()

        os.remove('temp1.txt')
        os.remove('temp2.txt')
    else:
        print ('\n\nLinear Regression Method is selected')
        # insert constants
        X_tr.insert(loc = 0 , column= 'constant', value = np.ones(trdata.shape[0]))
        X_tst.insert(loc = 0 , column= 'constant', value = np.ones(tstdata.shape[0]))

        # make beta_hat
        beta_hat = np.linalg.inv(X_tr.transpose() @ X_tr) @ np.array(X_tr).transpose() @ y_tr

        # make y_hat
        y_hat_tr = X_tr @ beta_hat

        # R-squre
        resid = y_tr - y_hat_tr
        sse = (resid**2).sum()
        sst = ((y_tr - y_tr.mean())**2).sum()
        r_sq = 1 - sse/sst
        r_sq

        # MSE
        MSE = sse/(len(resid)-len(X_tr.columns))


        # make txt file

        # Coefficient
        f = open(out_name,'w+')
        f.write('Coefficients\n')
        f.write('-------------\n')
        f.write(f'Constant: {round(beta_hat[0],3)}\n')
        for i in range(1,len(beta_hat)):
            f.write(f'Beta {i} : {round(beta_hat[i],3)}\n')

        #  summary
        f.write('\nModel Summary\n')
        f.write('-------------\n')
        f.write(f'R-square = {round(r_sq,4)}\n')
        f.write(f'MSE = {round(MSE,3)}')


        f.close()

    print('------------------------')
    print('file has been saved')
    print('result is\n\n')
    
    r = open(out_name,'r')
    for line in r.readlines():
        print(line)
    r.close()
    pass

In [3]:
Data_Analysis()

Enter the name of training data file [(ex) veh.dat] :  veh.dat
Enter the name of test data file [(ex) vehtest.dat] :  vehtest.dat
Select the data Seperator(1 = ' ' or 2 = ','):  2
Does the data have column header? (y/n) :  n
Enter the column position of the response variable : 
 [from 1 to 19] :  19
what kind of method do you want to run? (1 = LDA Classification, 2 = Linear Regression) :  1
Enter the output file name to export [(ex) result.txt] :  result.txt




LDA Classification Method is selected
------------------------
file has been saved
result is


Confusion Matrix (Training)

----------------------------------

		Predicted Class 



		1	2	3	4



Actual 	1	76	23	2	4



Class 	2	30	76	1	3



	3	2	1	106	1



	4	1	0	0	99





Model Summary (Training)

------------------------------

Overall accuracy = 0.84



Confusion Matrix (Test)

---------------------------------

		Predicted Class 



		1	2	3	4



Actual 	1	48	30	4	4



Class 	2	23	44	10	8



	3	0	0	85	1



	4	0	0	2	77





Model Summary (Test) 

---------------------------------

Overall accuracy = 0.756
